## Accelerate Inference: Neural Network Pruning

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.layers import *

print(tf.version.VERSION)

2.9.2


In [3]:
# untar
# !tar -xvzf dataset.tar.gz
# load train
train_images = pickle.load(open('/content/drive/MyDrive/dataset/train_images.pkl', 'rb'))
train_labels = pickle.load(open('/content/drive/MyDrive/dataset/train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('/content/drive/MyDrive/dataset/val_images.pkl', 'rb'))
val_labels = pickle.load(open('/content/drive/MyDrive/dataset/val_labels.pkl', 'rb'))

In [4]:
# Define the neural network architecture (don't change this)

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [5]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 25, 25, 32)        896       
                                                                 
 activation (Activation)     (None, 25, 25, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 11, 11, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                        

In [6]:
# you can use the default hyper-parameters for training, 
# and val accuracy ~59% after 25 epochs and > 63% after 50 epochs

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, batch_size=32, epochs=50, 
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


703/703 [==============================] - 12s 6ms/step - loss: 1.5244 - accuracy: 0.3036 - val_loss: 1.3709 - val_accuracy: 0.4044
Epoch 2/50
703/703 [==============================] - 4s 5ms/step - loss: 1.3536 - accuracy: 0.4214 - val_loss: 1.2784 - val_accuracy: 0.4697
Epoch 3/50
703/703 [==============================] - 4s 5ms/step - loss: 1.2833 - accuracy: 0.4606 - val_loss: 1.2429 - val_accuracy: 0.4586
Epoch 4/50
703/703 [==============================] - 4s 5ms/step - loss: 1.2416 - accuracy: 0.4836 - val_loss: 1.1736 - val_accuracy: 0.5137
Epoch 5/50
703/703 [==============================] - 4s 5ms/step - loss: 1.2093 - accuracy: 0.5028 - val_loss: 1.1485 - val_accuracy: 0.5259
Epoch 6/50
703/703 [==============================] - 4s 5ms/step - loss: 1.1743 - accuracy: 0.5214 - val_loss: 1.1151 - val_accuracy: 0.5493
Epoch 7/50
703/703 [==============================] - 4s 6ms/step - loss: 1.1423 - accuracy: 0.5371 - val_loss: 1.0902 - val_accuracy: 0.5572
Epoch 8/50
703/7

In [7]:
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 10ms/step - loss: 0.7984 - accuracy: 0.6978


In [8]:
# perform pruning here
# ThiNet

def get_layer_output(layer_name):  
  layer_output=model.get_layer(layer_name).output #get output to start a new model
  intermediate_model=tf.keras.models.Model(inputs=model.input,outputs=layer_output) 
  layer_output=intermediate_model.predict(train_images)
  return layer_output

conv2d_input = train_images
conv2d_1_input = get_layer_output('activation')
conv2d_2_input = get_layer_output('dropout')
conv2d_3_input = get_layer_output('activation_2')
dense_input = get_layer_output('flatten')
dense_1_input = get_layer_output('dropout_2')

# add padding
conv2d_input = np.pad(conv2d_input,[(0,0),(1,1),(1,1),(0,0)])
conv2d_2_input = np.pad(conv2d_2_input,[(0,0),(1,1),(1,1),(0,0)])

# weights have dimensions: k x k x c x num filterst
conv2d_weights = model.get_layer('conv2d').weights
conv2d_1_weights = model.get_layer('conv2d_1').weights
conv2d_2_weights = model.get_layer('conv2d_2').weights
conv2d_3_weights = model.get_layer('conv2d_3').weights
dense_weights = model.get_layer('dense').weights
dense_1_weights = model.get_layer('dense_1').weights

# # get the weights 
# weights = model.get_weights()

# # you can use set_weights() to set some weights to zero, e.g.,
# # some references for pruning techniques: https://arxiv.org/pdf/1810.05270v2.pdf, https://arxiv.org/pdf/2001.04062.pdf

# weights[7][:10]=0
# model.set_weights(weights)


703/703 [==============================] - 1s 2ms/step


In [9]:
# a = np.pad(np.array([[1,2,3],[1,2,3],[1,2,3]]), 1).reshape(1,5,5,1)
# a = np.append(a, a, axis = 0)
# a = tf.convert_to_tensor(a)
# weight = tf.convert_to_tensor(np.ones((3,3,1,32)))
def convolution(input, weight):
  model = models.Sequential()
  model.add(Conv2D(weight.shape[-1], (weight.shape[0],weight.shape[1]), input_shape=(input.shape[1:])))
  model.set_weights([weight, tf.convert_to_tensor(np.ones(weight.shape[-1]))])
  return model.predict(input)
# convolution(a, weight)

In [24]:
"""
for each layer:
Input: Training set {(ˆxi, yˆi)}, and compression rate r
Output: The subset of removed channels: T
1: T ← ∅; I ← {1, 2, . . . , C};
2: while |T| < C × (1 − r) do
3:   min value ← +∞;
4:   for each item i ∈ I do
5:      tmpT ← T ∪ {i};
6:     compute value from Eq. 6 using tmpT;
7:     if value < min value then
8:       min value ← value; min i ← i;
9:     end if
10:  end for
11:  move min i from I into T;
12:  end while

Eq 6:
sum over all input images and convolution locations_l and filters f (sum over all j in T, convolution of the jth channel in W_f and jth slice of x_l)**2

So I need:
input into this layer (output from the prev layer), weights of this layer
"""
# import copy
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

def get_T_for_layer(input, weight, r = 0.95): # for each conv layer
  T = []
  C = input.shape[-1]
  while len(T) < np.floor(C * (1-r)):
    min_val = float('inf')
    min_c = -1
    for c in range(C):
      if c not in T:
        tempT = T[:]
        tempT.append(c)
      else:
        continue

      # eq 6
      for idx, j in enumerate(tempT):
        print(idx)
        input_ = input[:,:,:,j].reshape(*input.shape[:-1], 1)
        weight_ = weight[0][:,:,j,:].reshape(weight[0].shape[0], weight[0].shape[1], 1, -1)
        if idx == 0:
          print('here')
          sum_over_T = convolution(input_, weight_)
        else:
          print('hi')
          sum_over_T += convolution(input_, weight_)
      eq_6 = np.sum(sum_over_T ** 2)

      if eq_6 < min_val:
        min_val = eq_6
        min_c = c
        try:
          weight_0 = weight[0].numpy()
        except AttributeError:
          weight_0 = weight[0]
        weight_0[:,:,j,:] = 0
        weight[0] = weight_0
    T.append(min_c)

  return T, weight

  


In [25]:
conv_layers_inputs = [conv2d_input, conv2d_1_input, conv2d_2_input, conv2d_3_input]
conv_layers_weights = [conv2d_weights, conv2d_1_weights, conv2d_2_weights, conv2d_3_weights]
layer_names = ['conv2d', 'conv2d_1', 'conv2d_2','conv2d_3']
for i in range(len(conv_layers_inputs)):
  T, weight = get_T_for_layer(conv_layers_inputs[i], conv_layers_weights[i])
  print(i, T)
  # model.get_layer(layer_names[i]).set_weights(weight)


Streaming output truncated to the last 5000 lines.
703/703 [==============================] - 1s 1ms/step
0
here
703/703 [==============================] - 1s 1ms/step
0
here
703/703 [==============================] - 1s 1ms/step
0
here
703/703 [==============================] - 1s 1ms/step
0
here
703/703 [==============================] - 1s 1ms/step
0
here
703/703 [==============================] - 1s 1ms/step
0
here
703/703 [==============================] - 1s 1ms/step
0
here
703/703 [==============================] - 1s 1ms/step
0
here
703/703 [==============================] - 1s 1ms/step
0
here
703/703 [==============================] - 1s 1ms/step
0
here
703/703 [==============================] - 1s 1ms/step
0
here
703/703 [==============================] - 1s 1ms/step
0
here
703/703 [==============================] - 1s 1ms/step
0
here
703/703 [==============================] - 1s 1ms/step
0
here
703/703 [==============================] - 1s 1ms/step
0
here
703/703 [==========

In [26]:
# evaluate again to see how the accuracy changes
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 3ms/step - loss: 3.5258 - accuracy: 0.2079


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
import copy
model2 = tf.keras.models.clone_model(model)
model2.set_weights(model.get_weights())
# model2.get_weights()
# # ['conv2d', 'conv2d_1', 'conv2d_2','conv2d_3']
conv2d_3_w = conv2d_3_weights[0].numpy()
conv2d_3_w[:,:,26,:]=0
conv2d_3_w[:,:,60,:]=0
conv2d_3_w[:,:,53,:]=0
conv2d_3_ws = copy.deepcopy(conv2d_3_weights)
conv2d_3_ws[0] = conv2d_3_w
model2.get_layer('conv2d_3').set_weights(conv2d_3_ws)

conv2d_2_w = conv2d_2_weights[0].numpy()
conv2d_2_w[:,:,13,:]=0
conv2d_2_ws = copy.deepcopy(conv2d_2_weights)
conv2d_2_ws[0] = conv2d_2_w
model2.get_layer('conv2d_2').set_weights(conv2d_2_ws)

conv2d_1_w = conv2d_1_weights[0].numpy()
conv2d_1_w[:,:,27,:]=0
conv2d_1_ws = copy.deepcopy(conv2d_1_weights)
conv2d_1_ws[0] = conv2d_1_w
model2.get_layer('conv2d_1').set_weights(conv2d_1_ws)


model2.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model2.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 5ms/step - loss: 0.8645 - accuracy: 0.6721


[0.8645018339157104, 0.6720792055130005]

In [63]:
# for dense layers
exclude_weights = np.argsort(np.sum(dense_input @ dense_weights[0], 0))[:int(np.floor(dense_weights[0].shape[-1]*0.4))]
d_ws = copy.deepcopy(dense_weights)
d_w = d_ws[0].numpy()
d_w[:,exclude_weights]=0
d_ws[0] = d_w
model2.get_layer('dense').set_weights(d_ws)
model2.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 5ms/step - loss: 0.9498 - accuracy: 0.6214


[0.9498492479324341, 0.6213861107826233]

In [58]:
exclude_weights_1 = np.argsort(np.sum(dense_1_input @ dense_1_weights[0], 0))[:int(np.floor(dense_1_weights[0].shape[-1]*0.18))]
d_1_ws = copy.deepcopy(dense_1_weights)
d_1_w = d_1_ws[0].numpy()
d_1_w[:,exclude_weights_1]=0
d_1_ws[0] = d_1_w
model2.get_layer('dense_1').set_weights(d_1_ws)
model2.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 5ms/step - loss: 0.8865 - accuracy: 0.6606


[0.8865399956703186, 0.6605940461158752]

In [66]:
# you need to save the model's weights, naming it 'my_model_weights.h5'
model2.save_weights("my_model_weights_1.h5")

# running this cell will immediately download a file called 'my_model_weights.h5'
from google.colab import files
files.download("my_model_weights_1.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [64]:
total_weights = 0
zero_weights = 0
all_model2_weights = model2.get_weights()
for weight in all_model2_weights:
  total_weights += weight.size
  zero_weights += np.sum(weight == 0)

In [65]:
(0.6214 + zero_weights/total_weights)/2

0.48904055449772565